## Uploading data to Google Storage

In [1]:
import os

os.environ["BUCKET_NAME"] = "founder-classifier-storage"

In [6]:
## create the bucket

!gsutil mb -l us gs://%BUCKET_NAME%

Creating gs://founder-classifier-storage/...


In [39]:
## upload the training data
!gsutil cp ../data/train.json gs://%BUCKET_NAME%

## upload the test data
!gsutil cp ../data/test.json gs://%BUCKET_NAME%

Copying file://..\data\train.json [Content-Type=application/json]...
/ [0 files][    0.0 B/  4.5 MiB]                                                
/ [0 files][263.4 KiB/  4.5 MiB]                                                
-
\
\ [0 files][  1.0 MiB/  4.5 MiB]                                                
|
| [0 files][  2.1 MiB/  4.5 MiB]                                                
/
-
- [0 files][  3.1 MiB/  4.5 MiB]                                                
\
\ [0 files][  4.1 MiB/  4.5 MiB]                                                
|
BadRequestException: 400 Invalid bucket name: '%BUCKET_NAME%'
Copying file://..\data\test.json [Content-Type=application/json]...
/ [0 files][    0.0 B/  1.1 MiB]                                                
/ [0 files][263.4 KiB/  1.1 MiB]                                                
-
\
\ [0 files][  1.1 MiB/  1.1 MiB]                                                
BadRequestException: 400 Invalid bucket name: '%BUCKET

## Download from Google Cloud Storage

In [16]:
from google.cloud import storage
client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket('founder-classifier-storage')
# Then do other things...
blob = bucket.get_blob('train.json')

blob.download_to_filename('./data/train.json')

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "My First Project-46b011a72a1d.json"

## Launch job

In [26]:
!gcloud ai-platform jobs submit training TEST \
--region europe-west1 \
--job-dir gs://founder-classifier-storage \
--package-path trainer/ \
--module-name trainer.task \
--

jobId: TEST
state: QUEUED


Job [TEST] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe TEST

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs TEST


In [ ]:
## https://github.com/tensorflow/models/tree/master/tutorials/image/cifar10_estimator

'''gcloud ml-engine jobs submit training cifarmultigpu \
    --runtime-version 1.2 \
    --job-dir=$MY_BUCKET/model_dirs/cifarmultigpu \
    --config cifar10_estimator/cmle_config.yaml \
    --package-path cifar10_estimator/ \
    --module-name cifar10_estimator.cifar10_main \
    -- \
    --data-dir=$MY_BUCKET/cifar-10-data \
    --num-gpus=4 \
    --train-steps=1000'''

In [ ]:
! gcloud ai-platform jobs stream-logs TEST

## train locally

In [34]:
import os

os.environ['MODULE_NAME'] =  'founder-classifier/trainer.task'
os.environ['JOB_DIR'] = 'output'
os.environ['PACKAGE_PATH'] = 'founder-classifier/trainer'

In [ ]:
gcloud ai-platform jobs submit training founder-classifier-test \
  --staging-bucket gs://founder-classifier-storage \
  --module-name trainer.task \
  --package-path ./trainer \
  --runtime-version 1.14 \
  --region europe-west1 \
  --python-version 3.5
  -- \
  --data-folder gs://founder-classifier-storage \
  --batch-size 32 \
  --learning-rate 0.001 \
  --steps 1000 

In [38]:
! gcloud ai-platform local train \
--module-name=%MODULE_NAME% \
--job-dir=%JOB_DIR% \
--package-path=%PACKAGE_PATH%

C:\Users\Aymeric\Miniconda3\python.exe: Error while finding module specification for 'founder-classifier/trainer.task' (ModuleNotFoundError: No module named 'founder-classifier/trainer')


locally

In [3]:
gcloud ai-platform local train --module-name=trainer.task --package-path=trainer -- --data-folder=gs://founder-classifier-storage

C:\Users\Aymeric\Miniconda3\python.exe: Error while finding module specification for 'trainer.task' (ModuleNotFoundError: No module named 'trainer')


cloud

In [ ]:
gcloud ai-platform jobs submit training founder_classifier_training_job_gpu --staging-bucket gs://founder-classifier-storage --module-name trainer.task --package-path ./trainer --runtime-version 1.14 --region europe-west1 --scale-tier basic-gpu -- --data-folder gs://founder-classifier-storage --batch-size 32  --learning-rate 0.001 --steps 1000 

In [37]:
gcloud ai-platform local train --module-name=trainer.task --package-path=trainer

SyntaxError: can't assign to operator (<ipython-input-37-f6f55a258210>, line 2)

## Deploy the model

https://medium.com/searce/creating-deep-learning-models-training-and-deploying-it-on-google-cloud-ml-engine-using-9a4ed6a84076

https://cloud.google.com/sdk/gcloud/reference/ai-platform/models/create

In [ ]:
gcloud ai-platform models create founder_classifier_model ^
--description "Model that classifies a picture of Jeff Bezos, Bill Gates or Larry Page" ^
--regions europe-west1

In [ ]:
gcloud ai-platform versions create "version_2" --model founder_classifier_model ^
--origin gs://founder-classifier-storage/model/1570723003 ^
--runtime-version 1.13 ^
--python-version 3.5

In [ ]:
gcloud ai-platform predict --model founder_classifier_model --version version_2 --json-instances image.json

In [34]:
from utils import from_image_to_json

from_image_to_json('jeff_bezos6.jpg')